In [ ]:
import pandas as pd
import numpy as np

## EDUCATION ##

# Chemin relatif vers le fichier education.csv
file_path = 'education-csv/education.csv'

# Lire le fichier en ignorant les premières lignes
df = pd.read_csv(file_path, skiprows=3)


# Supprimer la colonne "Indicator Code" qui semble être inutile
df.drop(columns=['Indicator Code'], inplace=True)

# Remplacer les noms de colonnes longs par des noms plus courts
new_columns = {
    'Country Name': 'Country',
    'Country Code': 'Code',
    'Indicator Name': 'Indicator'
}
df.rename(columns=new_columns, inplace=True)

# Remplacer les valeurs NaN par des chaînes vides
df.replace(to_replace=np.nan, value='', inplace=True)

# Filtrer les données pour obtenir celles spécifiques à Madagascar
madagascar_edu = df[df['Country'] == 'Madagascar']


## ECONOMY ##

# Chemin relatif vers le fichier pib.csv
file_path = 'economic-csv/pib.csv'

# Lire le fichier en ignorant les premières lignes
data = pd.read_csv(file_path, skiprows=3)


# Supprimer la colonne "Indicator Code" qui semble être inutile
data.drop(columns=['Indicator Code'], inplace=True)


# Remplacer les noms de colonnes longs par des noms plus courts
new_columns = {
    'Country Name': 'Country',
    'Country Code': 'Code',
    'Indicator Name': 'Indicator'
}
data.rename(columns=new_columns, inplace=True)

# Remplacer les valeurs NaN par des chaînes vides
data.replace(to_replace=np.nan, value='', inplace=True)

# Filtrer les données pour obtenir celles spécifiques à Madagascar
madagascar_pib = data[data['Country'] == 'Madagascar']

# Afficher les données spécifiques à Madagascar
print(madagascar_edu)
print(madagascar_pib)
